### Necessary imports

In [1]:
import SimpleITK as sitk
import tensorflow as tf
import os
import pandas as pd
import numpy as np

from dltk.io.augmentation import add_gaussian_noise, flip, extract_class_balanced_example_array
from dltk.io.preprocessing import whitening
from dltk.io.abstract_reader import Reader

from dltk.core.metrics import dice
from dltk.networks.segmentation.unet import residual_unet_3d

In [2]:
EVAL_EVERY_N_STEPS = 100
EVAL_STEPS = 1

NUM_CLASSES = 9
NUM_CHANNELS = 3

NUM_FEATURES_IN_SUMMARIES = min(4, NUM_CHANNELS)

BATCH_SIZE = 16
SHUFFLE_CACHE_SIZE = 64

MAX_STEPS = 50000

### Reader function

In [3]:
def read_fn(file_references, mode, params=None):
    """A custom python read function for interfacing with nii image files.
    Args:
        file_references (list): A list of lists containing file references, such
            as [['id_0', 'image_filename_0', target_value_0], ...,
            ['id_N', 'image_filename_N', target_value_N]].
        mode (str): One of the tf.estimator.ModeKeys strings: TRAIN, EVAL or
            PREDICT.
        params (dict, optional): A dictionary to parameterise read_fn ouputs
            (e.g. reader_params = {'n_examples': 10, 'example_size':
            [64, 64, 64], 'extract_examples': True}, etc.).
    Yields:
        dict: A dictionary of reader outputs for dltk.io.abstract_reader.
    """

    def _augment(img, lbl):
        """Image augmentation function"""
        img = add_gaussian_noise(img, sigma=0.1)
        [img, lbl] = flip([img, lbl], axis=1)

        return img, lbl

    for f in file_references:
        subject_id = f[0]
        img_fn = f[1]

        # Read the image nii with sitk and keep the pointer to the sitk.Image
        # of an input
        t1_sitk = sitk.ReadImage(str(os.path.join(img_fn, 'T1.nii')))
        t1 = sitk.GetArrayFromImage(t1_sitk)
        t1_ir = sitk.GetArrayFromImage(
            sitk.ReadImage(str(os.path.join(img_fn, 'T1_IR.nii'))))
        t2_fl = sitk.GetArrayFromImage(
            sitk.ReadImage(str(os.path.join(img_fn, 'T2_FLAIR.nii'))))

        # Normalise volume images to zero mean and unit variance
        t1 = whitening(t1)
        t1_ir = whitening(t1_ir)
        t2_fl = whitening(t2_fl)

        # Create a 4D multi-sequence image (i.e. [channels, x, y, z])
        images = np.stack([t1, t1_ir, t2_fl], axis=-1).astype(np.float32)

        if mode == tf.estimator.ModeKeys.PREDICT:
            yield {'features': {'x': images},
                   'labels': None,
                   'sitk': t1_sitk,
                   'subject_id': subject_id}

        lbl = sitk.GetArrayFromImage(sitk.ReadImage(str(os.path.join(
            img_fn,
            'LabelsForTraining.nii')))).astype(np.int32)

        # Augment if used in training mode
        if mode == tf.estimator.ModeKeys.TRAIN:
            images, lbl = _augment(images, lbl)

        # Check if the reader is supposed to return training examples or full
        #  images
        if params['extract_examples']:
            n_examples = params['n_examples']
            example_size = params['example_size']

            images, lbl = extract_class_balanced_example_array(
                image=images,
                label=lbl,
                example_size=example_size,
                n_examples=n_examples,
                classes=9)

            for e in range(n_examples):
                yield {'features': {'x': images[e].astype(np.float32)},
                       'labels': {'y': lbl[e].astype(np.int32)},
                       'subject_id': subject_id}
        else:
            yield {'features': {'x': images},
                   'labels': {'y': lbl},
                   'sitk': t1_sitk,
                   'subject_id': subject_id}

    return

### Model function

In [4]:
def model_fn(features, labels, mode, params):
    """Model function to construct a tf.estimator.EstimatorSpec. It creates a
        network given input features and training targets (labels). 
        Further, loss, optimiser, evaluation ops and
        custom tensorboard summary ops can be added. For additional information,
        please refer to https://www.tensorflow.org/api_docs/python/tf/estimator/Estimator#model_fn.
    Args:
        features (tf.Tensor): Tensor of input features to train from. Required
            rank and dimensions are determined by the subsequent ops
            (i.e. the network).
        labels (tf.Tensor): Tensor of training targets or labels. Required rank
            and dimensions are determined by the network output.
        mode (str): One of the tf.estimator.ModeKeys: TRAIN, EVAL or PREDICT
        params (dict, optional): A dictionary to parameterise the model_fn
            (e.g. learning_rate)
    Returns:
        tf.estimator.EstimatorSpec: A custom EstimatorSpec for this experiment
    """

    # 1. create a model and its outputs
    net_output_ops = residual_unet_3d(
        inputs=features['x'],
        num_classes=NUM_CLASSES,
        num_res_units=2,
        filters=(16, 32, 64, 128),
        strides=((1, 1, 1), (1, 2, 2), (1, 2, 2), (1, 2, 2)),
        mode=mode,
        kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-4))

    # 1.1 Generate predictions only (for `ModeKeys.PREDICT`)
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(
            mode=mode,
            predictions=net_output_ops,
            export_outputs={'out': tf.estimator.export.PredictOutput(net_output_ops)})

    # 2. set up a loss function
    ce = tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=net_output_ops['logits'],
        labels=labels['y'])
    loss = tf.reduce_mean(ce)

    # 3. define a training op and ops for updating moving averages
    # (i.e. for batch normalisation)
    global_step = tf.train.get_global_step()
    optimiser = tf.train.MomentumOptimizer(
        learning_rate=params["learning_rate"],
        momentum=0.9)

    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(update_ops):
        train_op = optimiser.minimize(loss, global_step=global_step)

    # 4.1 (optional) create custom image summaries for tensorboard
    my_image_summaries = {}
    my_image_summaries['feat_t1'] = features['x'][0, 0, :, :, 0]
    my_image_summaries['feat_t1_ir'] = features['x'][0, 0, :, :, 1]
    my_image_summaries['feat_t2_flair'] = features['x'][0, 0, :, :, 2]
    my_image_summaries['labels'] = tf.cast(labels['y'], tf.float32)[0, 0, :, :]
    my_image_summaries['predictions'] = tf.cast(net_output_ops['y_'], tf.float32)[0, 0, :, :]

    expected_output_size = [1, 128, 128, 1]  # [B, W, H, C]
    [tf.summary.image(name, tf.reshape(image, expected_output_size))
     for name, image in my_image_summaries.items()]

    # 4.2 (optional) create custom metric summaries for tensorboard
    dice_tensor = tf.py_func(dice, [net_output_ops['y_'],
                                    labels['y'],
                                    tf.constant(NUM_CLASSES)], tf.float32)
    [tf.summary.scalar('dsc_l{}'.format(i), dice_tensor[i])
     for i in range(NUM_CLASSES)]

    # 5. Return EstimatorSpec object
    return tf.estimator.EstimatorSpec(mode=mode,
                                      predictions=net_output_ops,
                                      loss=loss,
                                      train_op=train_op,
                                      eval_metric_ops=None)

### Train the network

In [5]:
def train(args):

    np.random.seed(42)
    tf.set_random_seed(42)
    
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    
    print('Setting up...')

    # Parse csv files for file names
    all_filenames = pd.read_csv(
        args.train_csv,
        dtype=object,
        keep_default_na=False,
        na_values=[]).as_matrix()

    train_filenames = all_filenames[1:4]
    val_filenames = all_filenames[4:5]

    # Set up a data reader to handle the file i/o.
    reader_params = {'n_examples': 18,
                     'example_size': [4, 128, 128],
                     'extract_examples': True}
    reader_example_shapes = {'features': {'x': reader_params['example_size'] + [NUM_CHANNELS, ]},
                             'labels': {'y': reader_params['example_size']}}
    reader = Reader(read_fn,
                    {'features': {'x': tf.float32},
                     'labels': {'y': tf.int32}})

    # Get input functions and queue initialisation hooks for training and
    # validation data
    train_input_fn, train_qinit_hook = reader.get_inputs(
        file_references=train_filenames,
        mode=tf.estimator.ModeKeys.TRAIN,
        example_shapes=reader_example_shapes,
        batch_size=BATCH_SIZE,
        shuffle_cache_size=SHUFFLE_CACHE_SIZE,
        params=reader_params)

    val_input_fn, val_qinit_hook = reader.get_inputs(
        file_references=val_filenames,
        mode=tf.estimator.ModeKeys.EVAL,
        example_shapes=reader_example_shapes,
        batch_size=BATCH_SIZE,
        shuffle_cache_size=SHUFFLE_CACHE_SIZE,
        params=reader_params)

    # Instantiate the neural network estimator
    nn = tf.estimator.Estimator(
        model_fn=model_fn,
        model_dir=args.model_path,
        params={"learning_rate": 0.001},
        config=tf.estimator.RunConfig(session_config=config))

    # Hooks for validation summaries
    val_summary_hook = tf.contrib.training.SummaryAtEndHook(
        os.path.join(args.model_path, 'eval'))
    step_cnt_hook = tf.train.StepCounterHook(
        every_n_steps=EVAL_EVERY_N_STEPS,
        output_dir=args.model_path)

    print('Starting training...')
    try:
        for _ in range(MAX_STEPS // EVAL_EVERY_N_STEPS):
            nn.train(
                input_fn=train_input_fn,
                hooks=[train_qinit_hook, step_cnt_hook],
                steps=EVAL_EVERY_N_STEPS)

            if args.run_validation:
                results_val = nn.evaluate(
                    input_fn=val_input_fn,
                    hooks=[val_qinit_hook, val_summary_hook],
                    steps=EVAL_STEPS)
                print('Step = {}; val loss = {:.5f};'.format(
                    results_val['global_step'], results_val['loss']))

    except KeyboardInterrupt:
        pass

    print('Stopping now.')
    export_dir = nn.export_savedmodel(
        export_dir_base=args.model_path,
        serving_input_receiver_fn=reader.serving_input_receiver_fn(reader_example_shapes))
    print('Model saved to {}.'.format(export_dir))

In [6]:
class objdict(dict):
    def __getattr__(self, name):
        if name in self:
            return self[name]
        else:
            raise AttributeError("No such attribute: " + name)

    def __setattr__(self, name, value):
        self[name] = value

    def __delattr__(self, name):
        if name in self:
            del self[name]
        else:
            raise AttributeError("No such attribute: " + name)

In [7]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Set up arguments
args = objdict({'run_validation': True,
       'restart': True,
       'verbose': True,
       'cuda_devices': '0',
       'model_path': '/vol/medic02/users/sk1712/DLTK/mrbrains_segmentation/',
       'train_csv': 'mrbrains.csv'})

# Set verbosity
if args.verbose:
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
    tf.logging.set_verbosity(tf.logging.INFO)
else:
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
    tf.logging.set_verbosity(tf.logging.ERROR)

# GPU allocation options
os.environ["CUDA_VISIBLE_DEVICES"] = args.cuda_devices

# Handle restarting and resuming training
if args.restart:
    print('Restarting training from scratch.')
    os.system('rm -rf {}'.format(args.model_path))

if not os.path.isdir(args.model_path):
    os.system('mkdir -p {}'.format(args.model_path))
else:
    print('Resuming training on model_path {}'.format(args.model_path))

# Call training
train(args)

Restarting training from scratch.
Setting up...
INFO:tensorflow:Using config: {'_model_dir': '/vol/medic02/users/sk1712/DLTK/mrbrains_segmentation/', '_num_worker_replicas': 1, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_train_distribute': None, '_global_id_in_cluster': 0, '_num_ps_replicas': 0, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f0837dbd3c8>, '_save_summary_steps': 100, '_master': '', '_tf_random_seed': None, '_session_config': gpu_options {
  allow_growth: true
}
, '_task_id': 0, '_evaluation_master': '', '_is_chief': True, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000}
Starting training...
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Init conv tensor shape (?, 4, 128, 128, 16)
INFO:tensorflow:Encoder at res_scale 1 tensor shape: (?, 4, 64, 64, 32)
INFO:tensorflow:Encoder at res_scale 2 tensor shape: (?, 4, 32, 32, 64)
I

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Init conv tensor shape (?, 4, 128, 128, 16)
INFO:tensorflow:Encoder at res_scale 1 tensor shape: (?, 4, 64, 64, 32)
INFO:tensorflow:Encoder at res_scale 2 tensor shape: (?, 4, 32, 32, 64)
INFO:tensorflow:Encoder at res_scale 3 tensor shape: (?, 4, 16, 16, 128)
INFO:tensorflow:Upsampling from (None, 4, 16, 16, 128) to [None, 4, 32, 32, 128]
INFO:tensorflow:Decoder at res_scale 2 tensor shape: (?, 4, 32, 32, 64)
INFO:tensorflow:Upsampling from (None, 4, 32, 32, 64) to [None, 4, 64, 64, 64]
INFO:tensorflow:Decoder at res_scale 1 tensor shape: (?, 4, 64, 64, 32)
INFO:tensorflow:Upsampling from (None, 4, 64, 64, 32) to [None, 4, 128, 128, 32]
INFO:tensorflow:Decoder at res_scale 0 tensor shape: (?, 4, 128, 128, 16)
INFO:tensorflow:Output tensor shape (?, 4, 128, 128, 9)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-05-22-17:47:24
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters fr

INFO:tensorflow:Saving checkpoints for 501 into /vol/medic02/users/sk1712/DLTK/mrbrains_segmentation/model.ckpt.
INFO:tensorflow:global_step/sec: 0.698926
INFO:tensorflow:loss = 0.32236123, step = 500
INFO:tensorflow:Saving checkpoints for 600 into /vol/medic02/users/sk1712/DLTK/mrbrains_segmentation/model.ckpt.
INFO:tensorflow:Loss for final step: 0.40891665.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Init conv tensor shape (?, 4, 128, 128, 16)
INFO:tensorflow:Encoder at res_scale 1 tensor shape: (?, 4, 64, 64, 32)
INFO:tensorflow:Encoder at res_scale 2 tensor shape: (?, 4, 32, 32, 64)
INFO:tensorflow:Encoder at res_scale 3 tensor shape: (?, 4, 16, 16, 128)
INFO:tensorflow:Upsampling from (None, 4, 16, 16, 128) to [None, 4, 32, 32, 128]
INFO:tensorflow:Decoder at res_scale 2 tensor shape: (?, 4, 32, 32, 64)
INFO:tensorflow:Upsampling from (None, 4, 32, 32, 64) to [None, 4, 64, 64, 64]
INFO:tensorflow:Decoder at res_scale 1 tensor shape: (?, 4, 64, 64, 32)
INFO:tensorflow:Upsamp

INFO:tensorflow:Output tensor shape (?, 4, 128, 128, 9)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /vol/medic02/users/sk1712/DLTK/mrbrains_segmentation/model.ckpt-800
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 801 into /vol/medic02/users/sk1712/DLTK/mrbrains_segmentation/model.ckpt.
INFO:tensorflow:global_step/sec: 0.704714
INFO:tensorflow:loss = 0.29166177, step = 800
INFO:tensorflow:Saving checkpoints for 900 into /vol/medic02/users/sk1712/DLTK/mrbrains_segmentation/model.ckpt.
INFO:tensorflow:Loss for final step: 0.25289825.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Init conv tensor shape (?, 4, 128, 128, 16)
INFO:tensorflow:Encoder at res_scale 1 tensor shape: (?, 4, 64, 64, 32)
INFO:tensorflow:Encoder at res_scale 2 tensor shape: (?, 4, 32, 32, 64)
INFO:tensorflow:Encoder at res

INFO:tensorflow:Decoder at res_scale 2 tensor shape: (?, 4, 32, 32, 64)
INFO:tensorflow:Upsampling from (None, 4, 32, 32, 64) to [None, 4, 64, 64, 64]
INFO:tensorflow:Decoder at res_scale 1 tensor shape: (?, 4, 64, 64, 32)
INFO:tensorflow:Upsampling from (None, 4, 64, 64, 32) to [None, 4, 128, 128, 32]
INFO:tensorflow:Decoder at res_scale 0 tensor shape: (?, 4, 128, 128, 16)
INFO:tensorflow:Output tensor shape (?, 4, 128, 128, 9)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /vol/medic02/users/sk1712/DLTK/mrbrains_segmentation/model.ckpt-1100
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1101 into /vol/medic02/users/sk1712/DLTK/mrbrains_segmentation/model.ckpt.
INFO:tensorflow:global_step/sec: 0.701077
INFO:tensorflow:loss = 0.25021148, step = 1100
INFO:tensorflow:Saving checkpoints for 1200 into 

...

INFO:tensorflow:Encoder at res_scale 1 tensor shape: (?, 4, 64, 64, 32)
INFO:tensorflow:Encoder at res_scale 2 tensor shape: (?, 4, 32, 32, 64)
INFO:tensorflow:Encoder at res_scale 3 tensor shape: (?, 4, 16, 16, 128)
INFO:tensorflow:Upsampling from (None, 4, 16, 16, 128) to [None, 4, 32, 32, 128]
INFO:tensorflow:Decoder at res_scale 2 tensor shape: (?, 4, 32, 32, 64)
INFO:tensorflow:Upsampling from (None, 4, 32, 32, 64) to [None, 4, 64, 64, 64]
INFO:tensorflow:Decoder at res_scale 1 tensor shape: (?, 4, 64, 64, 32)
INFO:tensorflow:Upsampling from (None, 4, 64, 64, 32) to [None, 4, 128, 128, 32]
INFO:tensorflow:Decoder at res_scale 0 tensor shape: (?, 4, 128, 128, 16)
INFO:tensorflow:Output tensor shape (?, 4, 128, 128, 9)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-05-23-13:17:09
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /vol/medic02/users/sk1712/DLTK/mrbrains_segmentation/model.ckpt-48900
INFO:tensorflow:Runni

INFO:tensorflow:global_step/sec: 0.693566
INFO:tensorflow:loss = 0.13807376, step = 49100
INFO:tensorflow:Saving checkpoints for 49200 into /vol/medic02/users/sk1712/DLTK/mrbrains_segmentation/model.ckpt.
INFO:tensorflow:Loss for final step: 0.14220928.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Init conv tensor shape (?, 4, 128, 128, 16)
INFO:tensorflow:Encoder at res_scale 1 tensor shape: (?, 4, 64, 64, 32)
INFO:tensorflow:Encoder at res_scale 2 tensor shape: (?, 4, 32, 32, 64)
INFO:tensorflow:Encoder at res_scale 3 tensor shape: (?, 4, 16, 16, 128)
INFO:tensorflow:Upsampling from (None, 4, 16, 16, 128) to [None, 4, 32, 32, 128]
INFO:tensorflow:Decoder at res_scale 2 tensor shape: (?, 4, 32, 32, 64)
INFO:tensorflow:Upsampling from (None, 4, 32, 32, 64) to [None, 4, 64, 64, 64]
INFO:tensorflow:Decoder at res_scale 1 tensor shape: (?, 4, 64, 64, 32)
INFO:tensorflow:Upsampling from (None, 4, 64, 64, 32) to [None, 4, 128, 128, 32]
INFO:tensorflow:Decoder at res_scale 0 tensor shap

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /vol/medic02/users/sk1712/DLTK/mrbrains_segmentation/model.ckpt-49400
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 49401 into /vol/medic02/users/sk1712/DLTK/mrbrains_segmentation/model.ckpt.
INFO:tensorflow:global_step/sec: 0.70082
INFO:tensorflow:loss = 0.1400936, step = 49400
INFO:tensorflow:Saving checkpoints for 49500 into /vol/medic02/users/sk1712/DLTK/mrbrains_segmentation/model.ckpt.
INFO:tensorflow:Loss for final step: 0.15080473.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Init conv tensor shape (?, 4, 128, 128, 16)
INFO:tensorflow:Encoder at res_scale 1 tensor shape: (?, 4, 64, 64, 32)
INFO:tensorflow:Encoder at res_scale 2 tensor shape: (?, 4, 32, 32, 64)
INFO:tensorflow:Encoder at res_scale 3 tensor shape: (?, 4, 16, 16, 128)
INFO:te

INFO:tensorflow:Decoder at res_scale 2 tensor shape: (?, 4, 32, 32, 64)
INFO:tensorflow:Upsampling from (None, 4, 32, 32, 64) to [None, 4, 64, 64, 64]
INFO:tensorflow:Decoder at res_scale 1 tensor shape: (?, 4, 64, 64, 32)
INFO:tensorflow:Upsampling from (None, 4, 64, 64, 32) to [None, 4, 128, 128, 32]
INFO:tensorflow:Decoder at res_scale 0 tensor shape: (?, 4, 128, 128, 16)
INFO:tensorflow:Output tensor shape (?, 4, 128, 128, 9)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /vol/medic02/users/sk1712/DLTK/mrbrains_segmentation/model.ckpt-49700
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 49701 into /vol/medic02/users/sk1712/DLTK/mrbrains_segmentation/model.ckpt.
INFO:tensorflow:global_step/sec: 0.699296
INFO:tensorflow:loss = 0.11960021, step = 49700
INFO:tensorflow:Saving checkpoints for 49800 i

Step = 50000; val loss = 0.22640;
Stopping now.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Init conv tensor shape (?, 4, 128, 128, 16)
INFO:tensorflow:Encoder at res_scale 1 tensor shape: (?, 4, 64, 64, 32)
INFO:tensorflow:Encoder at res_scale 2 tensor shape: (?, 4, 32, 32, 64)
INFO:tensorflow:Encoder at res_scale 3 tensor shape: (?, 4, 16, 16, 128)
INFO:tensorflow:Upsampling from (None, 4, 16, 16, 128) to [None, 4, 32, 32, 128]
INFO:tensorflow:Decoder at res_scale 2 tensor shape: (?, 4, 32, 32, 64)
INFO:tensorflow:Upsampling from (None, 4, 32, 32, 64) to [None, 4, 64, 64, 64]
INFO:tensorflow:Decoder at res_scale 1 tensor shape: (?, 4, 64, 64, 32)
INFO:tensorflow:Upsampling from (None, 4, 64, 64, 32) to [None, 4, 128, 128, 32]
INFO:tensorflow:Decoder at res_scale 0 tensor shape: (?, 4, 128, 128, 16)
INFO:tensorflow:Output tensor shape (?, 4, 128, 128, 9)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signat